# Lab | Data cleaning and wrangling

For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs.

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import scipy.stats as stats

In [2]:
df = pd.read_csv("/Users/ignaciolorenzoqueralt/Documents/Ironhack/labs/lab-cleaning-numerical-data/files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv")

In [3]:
def standardize_headers(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
standardize_headers(df)

### 1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

In [4]:
df=df.drop(['customer'], axis=1)

In [14]:
df.head(3)

,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,Suburban,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,Suburban,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,Suburban,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize


In [6]:
"""
def remove_outliers2(df, skip_columns=[]):
    in_columns=df.select_dtypes(np.number).columns
    for column in in_columns:
        if column not in skip_columns:
            low = .05
            high = .95
            quant_df = df.quantile([low, high])
    return quant_df.shape
"""

'\ndef remove_outliers2(df, skip_columns=[]):\n    in_columns=df.select_dtypes(np.number).columns\n    for column in in_columns:\n        if column not in skip_columns:\n            low = .05\n            high = .95\n            quant_df = df.quantile([low, high])\n    return quant_df.shape\n'

In [7]:
def remove_outliers(df, threshold=1.5, in_columns=df.select_dtypes(np.number).columns, skip_columns=[]):
    for column in in_columns:
        if column not in skip_columns:
            upper = np.percentile(df[column],75)
            lower = np.percentile(df[column],25)
            iqr = upper - lower
            upper_limit = upper + (threshold * iqr)
            lower_limit = lower - (threshold * iqr)
            df = df[(df[column]>lower_limit) & (df[column]<upper_limit)]
    return df


In [8]:
df = remove_outliers(df)
df

IndexError: cannot do a non-empty take from an empty axes.

### 2. Create a copy of the dataframe for the data wrangling.

In [9]:
df_copy = df.copy()
df_copy.shape

(9134, 23)

### 3. Normalize the continuous variables. You can use any one method you want.

In [10]:
def boxcox_transform(df):
    numeric_cols = df.select_dtypes(np.number).columns
    _ci = {column: None for column in numeric_cols}
    for column in numeric_cols:
        # since i know any columns should take negative numbers, to avoid -inf in df
        df[column] = np.where(df[column]<=0, np.NAN, df[column]) 
        df[column] = df[column].fillna(df[column].mean())
        transformed_data, ci = stats.boxcox(df[column])
        df[column] = transformed_data
        _ci[column] = [ci] 
    return df, _ci

In [11]:
df_copy, _ci = boxcox_transform(df_copy)
df_copy.shape

(9134, 23)

### 4. Encode the categorical variables

In [12]:
df_copy_cat = df_copy.select_dtypes('object')

In [13]:
df_copy_cat = pd.get_dummies(df_copy_cat, drop_first=True)

### 5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

In [16]:
int(df_copy['effective_to_date'])

TypeError: cannot convert the series to <class 'int'>

### 6. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.